In [1]:
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch

from pytorch_lightning.trainer import Trainer

from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.datasets.memory_dataset import MemoryMapDataset
from ptls.nn import TrxEncoder
from ptls.frames import PtlsDataModule

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
    
from nn.seq_encoder import ConvSeqEncoder
from modules.ts2vec_module import TS2Vec
from datasets.ts2vec_dataset import TS2VecDataset
from utils.encode import encode_data

d:\ProgramData\miniconda3\envs\ts2vec\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [2]:
df = pd.read_parquet("data/preprocessed_new/churn.parquet")
df.head()

,user_id,mcc_code,timestamp,amount,global_target,holiday_target,weekend_target,churn_target
0,0,19,2017-10-21 00:00:00,5023.0,0,0,1,0
1,0,2,2017-10-12 12:24:07,20000.0,0,0,0,0
2,0,10,2017-12-05 00:00:00,767.0,0,0,0,0
3,0,1,2017-10-21 00:00:00,2031.0,0,0,1,0
4,0,9,2017-10-24 13:14:24,36562.0,0,0,0,0


## Prepare dataset and dataloaders

In [3]:
preprocessor = PandasDataPreprocessor(
    col_id="user_id",
    col_event_time="timestamp",
    event_time_transformation="dt_to_timestamp",
    cols_category=["mcc_code"],
    cols_first_item=["global_target"]
)

In [4]:
data = preprocessor.fit_transform(df)

In [6]:
val_size = 0.1
test_size = 0.1

train, val_test = train_test_split(data, test_size=test_size+val_size, random_state=42)
val, test = train_test_split(val_test, test_size=test_size/(test_size+val_size), random_state=42)

In [7]:
train_ds = TS2VecDataset(train, min_seq_len=15)
val_ds = TS2VecDataset(val, min_seq_len=15)
test_ds = TS2VecDataset(test, min_seq_len=15)

In [8]:
datamodule = PtlsDataModule(
    train_data=train_ds,
    valid_data=val_ds,
    train_batch_size=16,
    valid_batch_size=16
)

## Model training

In [9]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 345, "out": 24}
    },
    numeric_values={
        "amount": "identity"
    }
)

seq_encoder = ConvSeqEncoder(
    trx_encoder,
    is_reduce_sequence=False,
    hidden_size=320,
    num_layers=10,
    dropout=0.1,
)

In [13]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.99, patience=50)
optimizer_partial = partial(torch.optim.Adam, lr=1e-3)

model = TS2Vec(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [14]:
trainer = Trainer(
    max_epochs=50,
    accelerator="gpu"
)

trainer.fit(model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                        | Params
-------------------------------------------------------------
0 | _loss        | HierarchicalContrastiveLoss | 0     
1 | _seq_encoder | ConvSeqEncoder              | 386 K 
2 | valid_loss   | MeanMetric                  | 0     
-------------------------------------------------------------
386 K     Trainable params
0         Non-trainable params
386 K     Total params
1.546     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

d:\ProgramData\miniconda3\envs\ts2vec\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


d:\ProgramData\miniconda3\envs\ts2vec\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 49: 100%|██████████| 249/249 [00:09<00:00, 25.01it/s, loss=1.25, v_num=30, seq_len=15.00, valid_loss=1.450]


## Model evaluation

In [15]:
train_val_ds = TS2VecDataset(train + val, min_seq_len=15)

In [16]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

In [18]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

y_pred = logreg.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred)

0.675554426951075